In [2]:
import torch
import esm
import biotite.structure.io as bsio
import os
from Bio.PDB import PDBParser, Superimposer
import numpy as np

model = esm.pretrained.esmfold_v1()
model = model.eval().cuda()

ModuleNotFoundError: No module named 'openfold'

In [ ]:
S_ref = 'SVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENL'
S_pred = 'SVPSQKTYHGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENL'

In [ ]:
def mse_pdb(pdb_file1, pdb_file2):
    # 创建PDB解析器对象
    parser = PDBParser()

    # 解析PDB文件
    structure1 = parser.get_structure('protein1', pdb_file1)
    structure2 = parser.get_structure('protein2', pdb_file2)

    # 获取第一个模型的第一个链
    model1 = structure1[0]
    chain1 = model1['A']

    # 获取第二个模型的第一个链
    model2 = structure2[0]
    chain2 = model2['A']
    
    atoms1 = [atom for atom in chain1.get_atoms() if atom.get_name() == 'CA']
    atoms2 = [atom for atom in chain2.get_atoms() if atom.get_name() == 'CA']
    # 创建超立方体对象
    superimposer = Superimposer()

    # 将链的原子坐标转换为列表
    # atoms1 = list(chain1.get_atoms())
    # atoms2 = list(chain2.get_atoms())

    # 对两个链进行超立方体对齐
    superimposer.set_atoms(atoms1, atoms2)

    # 计算均方差
    rmsd = superimposer.rms

    return rmsd

def cal_MSE(S_ref, S_pred):
    with torch.no_grad():
        ori_pdb = model.infer_pdb(S_ref)
        mut_pdb = model.infer_pdb(S_pred)
    with open("./ori.pdb", "w") as f:
        f.write(ori_pdb)
    with open("./mut.pdb", "w") as f:
        f.write(mut_pdb)
    pymol.cmd.reset()
    ref='./ori.pdb'
    input='./mut.pdb'
    pymol.cmd.load(ref, "ref")
    pymol.cmd.load(input, "input")
    rms = pymol.cmd.align("ref","input")
    print(rms)
    pymol.cmd.save("./output.pdb", "input")
    pymol.cmd.delete("ref")
    pymol.cmd.delete("input")    
    pdb_file1 = './output.pdb'
    pdb_file2 = './ori.pdb'
    mse = mse_pdb(pdb_file1, pdb_file2)
    print("MSE:", mse)
    os.remove("./output.pdb")
    os.remove("./ori.pdb")
    os.remove("./mut.pdb")
cal_MSE(S_ref, S_pred)